In [19]:
from importlib import reload

import myutman
reload(myutman.stand_utils)
reload(myutman.stand)
reload(myutman.window_algo)
reload(myutman.generation)
reload(myutman.fuse)

from myutman.fuse import FuseForWindowAlgo
from myutman.generation import ClientTerminalsReorderSampleGeneration, ChangeWithClientSampleGeneration, \
    ChangeWithTerminalSampleGeneration, ChangeSampleGeneration, StillSampleGeneration, \
    SimpleMultichangeSampleGeneration
from myutman.node_distribution import RoundrobinNodeDistribution, DependentNodeDistribution, \
    SecondMetaDependentNodeDistribution
from myutman.stand import Stand
from myutman.stand_utils import compare_vals, show_boxplots
from myutman.window_algo import WindowStreamingAlgo

import json

In [23]:
big_windows = [(30, 30), (45, 45), (60, 60)]
small_windows = [(6, 6), (9, 9), (12, 12)]

n_nodes = 5
stand_centralized = Stand(
    n_nodes=1,
    algo=WindowStreamingAlgo,
    client_node_distribution=RoundrobinNodeDistribution,
    terminal_node_distribution=RoundrobinNodeDistribution,
    fuse=FuseForWindowAlgo(),
    client_algo_kwargs={"window_sizes": big_windows},
    terminal_algo_kwargs={"window_sizes": big_windows}
)
stand_roundrobins = Stand(
    n_nodes=n_nodes,
    algo=WindowStreamingAlgo,
    client_node_distribution=RoundrobinNodeDistribution,
    terminal_node_distribution=RoundrobinNodeDistribution,
    fuse=FuseForWindowAlgo(),
    client_algo_kwargs={"window_sizes": small_windows},
    terminal_algo_kwargs={"window_sizes": small_windows}
)
stand_client_dependent = Stand(
    n_nodes=n_nodes,
    algo=WindowStreamingAlgo,
    client_node_distribution=DependentNodeDistribution,
    terminal_node_distribution=SecondMetaDependentNodeDistribution,
    fuse=FuseForWindowAlgo(),
    client_algo_kwargs={"window_sizes": big_windows},
    terminal_algo_kwargs={"window_sizes": small_windows}
)
stand_terminal_dependent = Stand(
    n_nodes=n_nodes,
    algo=WindowStreamingAlgo,
    client_node_distribution=SecondMetaDependentNodeDistribution,
    terminal_node_distribution=DependentNodeDistribution,
    fuse=FuseForWindowAlgo(),
    client_algo_kwargs={"window_sizes": small_windows},
    terminal_algo_kwargs={"window_sizes": big_windows}
)
"""stand_resharding = Stand(
    n_nodes=n_nodes,
    algo=WindowStreamingAlgo,
    client_node_distribution=DependentNodeDistribution,
    terminal_node_distribution=DependentNodeDistribution,
    fuse=FuseForWindowAlgo()
)"""

'stand_resharding = Stand(\n    n_nodes=n_nodes,\n    algo=WindowStreamingAlgo,\n    client_node_distribution=DependentNodeDistribution,\n    terminal_node_distribution=DependentNodeDistribution,\n    fuse=FuseForWindowAlgo()\n)'

In [24]:
print(stand_centralized.result_filename)

outputs/algo=WindowStreamingAlgo_client_dist=RoundrobinNodeDistribution_terminal_dist=RoundrobinNodeDistribution_nnodes=1


In [ ]:
stands = [
    stand_centralized,
    stand_roundrobins,
    stand_client_dependent,
    stand_terminal_dependent
]

generations = [
    ChangeSampleGeneration
]

n_clients = 5
n_terminals = 5

results = [[[] for _ in stands] for _ in generations]
for state in range(100):
    for i, generation in enumerate(generations):
        sample, change_points, change_points_ids = generation(state=state)(
            size=20000, 
            n_clients=n_clients,
            n_terminals=n_terminals,
            change_period=400,
            change_period_noise=0,
            change_interval=400
        )
        for j, stand in enumerate(stands):
            result1 = stand.test(
                p=0.05,
                sample=sample,
                change_points=change_points,
                change_ids=change_points_ids,
                n_clients=n_clients,
                n_terminals=n_terminals
            )
            print(result1)
            results[i][j].append(result1)
        with open('tuned_sizes_results.json', 'w') as output_file:
            json.dump(results, output_file, indent=4, ensure_ascii=False)


Result written to outputs/algo=WindowStreamingAlgo_client_dist=RoundrobinNodeDistribution_terminal_dist=RoundrobinNodeDistribution_nnodes=1_1583363674.504991.json
{'client_TDR': 1.0, 'client_MDR': 0.0, 'client_FDR': 0.3421052631578947, 'client_CDR': 1.75, 'terminal_TDR': 1.0, 'terminal_MDR': 0.0, 'terminal_FDR': 0.3142857142857143, 'terminal_CDR': 2.56, 'client_False': 13, 'terminal_False': 11, 'client_latency': 47.36, 'terminal_latency': 70.79166666666667}



Result written to outputs/algo=WindowStreamingAlgo_client_dist=RoundrobinNodeDistribution_terminal_dist=RoundrobinNodeDistribution_nnodes=5_1583363740.815109.json
{'client_TDR': 0.96, 'client_MDR': 0.040000000000000036, 'client_FDR': 0.41463414634146345, 'client_CDR': 0.9583333333333334, 'terminal_TDR': 1.0, 'terminal_MDR': 0.0, 'terminal_FDR': 0.3513513513513513, 'terminal_CDR': 1.6, 'client_False': 17, 'terminal_False': 13, 'client_latency': 79.25, 'terminal_latency': 106.33333333333333}



Result written to outputs/algo=WindowStreamingAlgo_client_dist=DependentNodeDistribution_terminal_dist=SecondMetaDependentNodeDistribution_nnodes=5_1583363874.163281.json
{'client_TDR': 1.0, 'client_MDR': 0.0, 'client_FDR': 0.3421052631578947, 'client_CDR': 1.75, 'terminal_TDR': 0.9583333333333334, 'terminal_MDR': 0.04166666666666663, 'terminal_FDR': 0.54, 'terminal_CDR': 4.8, 'client_False': 13, 'terminal_False': 27, 'client_latency': 47.36, 'terminal_latency': 83.47826086956522}



Result written to outputs/algo=WindowStreamingAlgo_client_dist=SecondMetaDependentNodeDistribution_terminal_dist=DependentNodeDistribution_nnodes=5_1583363986.100519.json
{'client_TDR': 0.96, 'client_MDR': 0.040000000000000036, 'client_FDR': 0.4545454545454546, 'client_CDR': 3.7083333333333335, 'terminal_TDR': 1.0, 'terminal_MDR': 0.0, 'terminal_FDR': 0.3142857142857143, 'terminal_CDR': 2.56, 'client_False': 20, 'terminal_False': 11, 'client_latency': 82.79166666666667, 'terminal_latency': 70.79166666666667}



Result written to outputs/algo=WindowStreamingAlgo_client_dist=RoundrobinNodeDistribution_terminal_dist=RoundrobinNodeDistribution_nnodes=1_1583364149.404806.json
{'client_TDR': 0.96, 'client_MDR': 0.040000000000000036, 'client_FDR': 0.3513513513513513, 'client_CDR': 1.6666666666666667, 'terminal_TDR': 1.0, 'terminal_MDR': 0.0, 'terminal_FDR': 0.3846153846153846, 'terminal_CDR': 2.44, 'client_False': 13, 'terminal_False': 15, 'client_latency': 45.375, 'terminal_latency': 68.0}


...


Result written to outputs/algo=WindowStreamingAlgo_client_dist=DependentNodeDistribution_terminal_dist=SecondMetaDependentNodeDistribution_nnodes=5_1583393535.65577.json
{'client_TDR': 1.0, 'client_MDR': 0.0, 'client_FDR': 0.3421052631578947, 'client_CDR': 2.125, 'terminal_TDR': 1.0, 'terminal_MDR': 0.0, 'terminal_FDR': 0.5, 'terminal_CDR': 4.96, 'client_False': 13, 'terminal_False': 24, 'client_latency': 62.24, 'terminal_latency': 57.125}



Result written to outputs/algo=WindowStreamingAlgo_client_dist=SecondMetaDependentNodeDistribution_terminal_dist=DependentNodeDistribution_nnodes=5_1583393630.484981.json
{'client_TDR': 1.0, 'client_MDR': 0.0, 'client_FDR': 0.4565217391304348, 'client_CDR': 4.0, 'terminal_TDR': 0.9583333333333334, 'terminal_MDR': 0.04166666666666663, 'terminal_FDR': 0.39473684210526316, 'terminal_CDR': 3.48, 'client_False': 21, 'terminal_False': 15, 'client_latency': 54.08, 'terminal_latency': 78.69565217391305}



Result written to outputs/algo=WindowStreamingAlgo_client_dist=RoundrobinNodeDistribution_terminal_dist=RoundrobinNodeDistribution_nnodes=1_1583393778.963163.json
{'client_TDR': 0.96, 'client_MDR': 0.040000000000000036, 'client_FDR': 0.41463414634146345, 'client_CDR': 2.0416666666666665, 'terminal_TDR': 0.9583333333333334, 'terminal_MDR': 0.04166666666666663, 'terminal_FDR': 0.39473684210526316, 'terminal_CDR': 2.72, 'client_False': 17, 'terminal_False': 15, 'client_latency': 41.041666666666664, 'terminal_latency': 103.65217391304348}



Result written to outputs/algo=WindowStreamingAlgo_client_dist=RoundrobinNodeDistribution_terminal_dist=RoundrobinNodeDistribution_nnodes=5_1583393843.640427.json
{'client_TDR': 1.0, 'client_MDR': 0.0, 'client_FDR': 0.3055555555555556, 'client_CDR': 1.125, 'terminal_TDR': 0.9583333333333334, 'terminal_MDR': 0.04166666666666663, 'terminal_FDR': 0.34285714285714286, 'terminal_CDR': 1.36, 'client_False': 11, 'terminal_False': 12, 'client_latency': 77.28, 'terminal_latency': 104.78260869565217}



Result written to outputs/algo=WindowStreamingAlgo_client_dist=DependentNodeDistribution_terminal_dist=SecondMetaDependentNodeDistribution_nnodes=5_1583393960.370525.json
{'client_TDR': 0.96, 'client_MDR': 0.040000000000000036, 'client_FDR': 0.41463414634146345, 'client_CDR': 2.0416666666666665, 'terminal_TDR': 0.875, 'terminal_MDR': 0.125, 'terminal_FDR': 0.5714285714285714, 'terminal_CDR': 5.04, 'client_False': 17, 'terminal_False': 28, 'client_latency': 41.041666666666664, 'terminal_latency': 65.42857142857143}


In [ ]:
show_boxplots('client_MDR', results[0], ['centralized', 'roundrobin', 'client_dependent', 'terminal_dependent'])

In [ ]:
show_boxplots('client_FDR', results[0], ['centralized', 'roundrobin', 'client_dependent', 'terminal_dependent'])

In [ ]:
show_boxplots('terminal_MDR', results[0], ['centralized', 'roundrobin', 'client_dependent', 'terminal_dependent'])

In [ ]:
show_boxplots('terminal_FDR', results[0], ['centralized', 'roundrobin', 'client_dependent', 'terminal_dependent'])

In [ ]:
show_boxplots('client_latency', results[0], ['centralized', 'roundrobin', 'client_dependent', 'terminal_dependent'])

In [ ]:
show_boxplots('terminal_latency', results[0], ['centralized', 'roundrobin', 'client_dependent', 'terminal_dependent'])

In [ ]:
stands = [
    stand_centralized,
    stand_roundrobins,
    stand_client_dependent,
]

generations = [
    SimpleMultichangeSampleGeneration
]

n_clients = 1
n_terminals = 1

results = [[[] for _ in stands] for _ in generations]
for state in range(1):
    for i, generation in enumerate(generations):
        sample, change_points, change_points_ids = generation(state=state)(
            size=10000,
            tau=200
        )
        for j, stand in enumerate(stands):
            result1 = stand.test(
                p=0.05,
                sample=sample,
                change_points=change_points,
                change_ids=change_points_ids,
                n_clients=n_clients,
                n_terminals=n_terminals
            )
            print(result1)
            results[i][j].append(result1)

In [ ]:
show_boxplots('client_MDR', results[0], ['centralized', 'roundrobin', 'client_dependent'])

In [ ]:
show_boxplots('client_FDR', results[0], ['centralized', 'roundrobin', 'client_dependent'])

In [ ]:
show_boxplots('client_latency', results[0], ['centralized', 'roundrobin', 'client_dependent'])

In [ ]:
with open('simple_results.json', 'w') as simple_file:
    json.dump(results, simple_file, indent=4, ensure_ascii=False)